In [1]:
from leap.example.simple_ea import simple_ea
from leap import core, real, operate as op

def ea(problem, initialize, probes=[], step_notify=[], evals=200, l=2):
    pop_size=5
    mutate_prob=1/l
    mutate_std=0.5

    ea = simple_ea(evals=evals, pop_size=pop_size,
                   individual_cls=core.Individual,  # Use the standard Individual as the prototype for the population.
                   decoder=core.IdentityDecoder(),  # Genotype and phenotype are the same for this task.
                   problem=problem,
                   evaluate=op.evaluate,  # Evaluate fitness with the basic evaluation operator.

                   initialize=initialize,

                   step_notify_list=step_notify,

                   # The operator pipeline.
                   pipeline=probes + [
                       # Select mu parents via tournament selection.
                       op.tournament(n=pop_size),
                       # Clone them to create offspring.
                       op.cloning,
                       # Apply Gaussian mutation to each gene with a certain probability.
                       op.mutate_gaussian(prob=mutate_prob, std=mutate_std, hard_bounds=(-5.12, 5.12))
                   ])
    return ea

ModuleNotFoundError: No module named 'leap.example'

In [ ]:
%matplotlib notebook
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from leap import real

problem1 = real.Spheroid()
problem2 = real.Rastrigin(a=10)
problem3 = real.NoisyQuartic()
problems = [problem1, problem2, problem3]

plt.figure(figsize=(8, 2))
plt.subplot(131, projection='3d')
real.plot_2d_problem(problem1, ax=plt.gca(), xlim=problem1.bounds, ylim=problem1.bounds);
plt.subplot(132, projection='3d')
real.plot_2d_problem(problem2, ax=plt.gca(), xlim=problem2.bounds, ylim=problem2.bounds);
plt.subplot(133, projection='3d')
real.plot_2d_problem(problem3, ax=plt.gca(), xlim=problem3.bounds, ylim=problem3.bounds);

In [ ]:
%matplotlib notebook
from leap.probe import PlotTrajectoryProbe, PlotProbe

def probe_plots(problems, modulo):
    plt.figure(figsize=(8, 3*len(problems)))
    trajectory_probes = []
    fitness_probes = []
    for i, p in enumerate(problems):
        plt.subplot(len(problems), 2, 2*i + 1)
        tp = PlotTrajectoryProbe(contours=p, xlim=p.bounds, ylim=p.bounds, granularity=0.1, modulo=modulo, ax=plt.gca())
        trajectory_probes.append(tp)
        
        plt.subplot(len(problems), 2, 2*i + 2)
        fp = PlotProbe(ylim=(0, 1), modulo=modulo, ax=plt.gca())
        fitness_probes.append(fp)
        
    return trajectory_probes, fitness_probes

trajectory_probes, fitness_probes = probe_plots([problem1, problem2, problem3], 50)

In [ ]:
from leap.contrib.transfer.sequential import PopulationSeedingRepertoire

def ea1(problem, initialize):
    return ea(problem, initialize,
              probes=[trajectory_probes[0], plot_probes[0]],
              step_notify=[trajectory_probes[0].set_step, plot_probes[0].set_step], evals=200, l=2)

random_init = real.create_real_value_sequence(bounds=[[-0.5, 0.5]] * 2)
    
rep = PopulationSeedingRepertoire(problems, random_init, ea1)